# 第1章: 基本的なプロンプト構造

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
%pip install anthropic --quiet

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Import python's built-in regular expression library
import re
from anthropic import AnthropicBedrock

%store -r MODEL_NAME
%store -r AWS_REGION

client = AnthropicBedrock(aws_region=AWS_REGION)

def get_completion(prompt, system=''):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": prompt}
        ],
        system=system
    )
    return message.content[0].text

---

## レッスン

AnthropicはレガシーAPI（旧版）である[Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html)と現在の[Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)の2つのAPIを提供しています。このチュートリアルでは、Messages APIのみを使用します。

Messages APIを使用してClaudeを呼び出すには、最低限以下のパラメータが必要です：
- `model`: 呼び出すモデルの[APIモデル名](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`: 停止する前に生成するトークンの最大数。Claudeはこの最大値に達する前に停止する場合があることに注意してください。このパラメータは生成するトークンの絶対最大数のみを指定します。さらに、これは*ハード*ストップであり、Claudeが単語や文の途中で生成を停止する可能性があります。

- `messages`: 入力メッセージの配列。私たちのモデルは、`user`（ユーザー）と`assistant`（アシスタント）の会話ターンが交互に行われるように訓練されています。新しい`Message`を作成する際、messagesパラメータで以前の会話ターンを指定すると、モデルは会話の次の`Message`を生成します。
  - 各入力メッセージは`role`と`content`を持つオブジェクトである必要があります。単一の`user`ロールメッセージを指定することも、複数の`user`と`assistant`メッセージを含めることもできます（その場合は交互である必要があります）。最初のメッセージは常に`user`ロールを使用する必要があります。

以下のようなオプションパラメータもあります：
- `system`: システムプロンプト - 詳細は以下で説明します。
  
- `temperature`: Claudeの応答の変動度。これらのレッスンと演習では、`temperature`を0に設定しています。

すべてのAPIパラメータの完全なリストについては、[APIドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html)をご覧ください。

### 例

正しくフォーマットされたプロンプトに対してClaudeがどのように応答するかを見てみましょう。以下の各セルについて、セルを実行（`shift+enter`）すると、Claudeの応答がブロックの下に表示されます。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

次に、正しいMessages APIフォーマットが含まれていないプロンプトを見てみましょう。これらの誤ったフォーマットのプロンプトに対して、Messages APIはエラーを返します。

まず、`messages`配列に`role`と`content`フィールドが欠けているMessages API呼び出しの例を示します。

> ⚠️ **警告：** プロンプトのmessagesパラメータの書式が正しくないため、以下のセルはエラーを返します。これは期待される動作です。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

以下は、`user`と`assistant`ロールが交互になっていないプロンプトです。

> ⚠️ **警告：** `user`と`assistant`ロールが交互になっていないため、Claudeはエラーメッセージを返します。これは期待される動作です。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user`と`assistant`メッセージは**必ず交互に**配置し、メッセージは**必ず`user`ターンで開始する**必要があります。プロンプト内に複数の`user`と`assistant`のペアを含めることもできます（マルチターン会話をシミュレートするかのように）。また、終端の`assistant`メッセージに単語を配置して、Claudeがあなたが中断したところから続きを生成するようにすることもできます（詳細は後の章で説明します）。

#### システムプロンプト

**システムプロンプト**も使用できます。システムプロンプトは、「User」ターンで質問やタスクを提示する前に、**Claudeにコンテキスト、指示、ガイドラインを提供**する方法です。

構造的には、システムプロンプトは`user`と`assistant`メッセージのリストとは別に存在し、そのため別の`system`パラメータに属します（ノートブックの[セットアップ](#setup)セクションにある`get_completion`ヘルパー関数の構造を参照してください）。

このチュートリアル内でシステムプロンプトを利用する場合は、completion関数に`system`フィールドを用意しています。システムプロンプトを使用したくない場合は、単に`SYSTEM_PROMPT`変数を空の文字列に設定してください。

#### システムプロンプトの例

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

なぜシステムプロンプトを使用するのでしょうか？**適切に書かれたシステムプロンプトは、様々な方法でClaudeのパフォーマンスを向上させることができます**。例えば、ルールや指示に従うClaudeの能力を向上させることができます。詳細については、Claudeでの[システムプロンプトの使用方法](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)に関するドキュメントをご覧ください。

次は演習に進みます。上記の内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの最下部にある[**サンプル実験場**](#example-playground)にスクロールしてください。

---

## 演習
- [演習 1.1 - 3まで数える](#exercise-11---counting-to-three)
- [演習 1.2 - システムプロンプト](#exercise-12---system-prompt)

### 演習 1.1 - 3まで数える
適切な`user` / `assistant`フォーマットを使用して、以下の`PROMPT`を編集してClaudeに**3まで数える**ように指示してください。出力では、あなたの解答が正しいかどうかも示されます。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_1_hint)

### 演習 1.2 - システムプロンプト

`SYSTEM_PROMPT`を修正して、Claudeが3歳の子どものように応答するようにしてください。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_2_hint)

### おめでとうございます！

この時点ですべての演習を解けていれば、次の章に進む準備ができています。プロンプト作成を楽しんでください！

---

## サンプル実験場

これは、このレッスンで示されたプロンプトサンプルを自由に試し、プロンプトを調整してClaudeの応答にどのような影響があるかを確認するためのエリアです。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))